### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete_df = pd.merge(student_data_df, school_data_df, how="left", on=["school_name", "school_name"])
#school_data_df.head(15)
len(student_data_df)#.head(15)
#school_data_complete_df.head()

39170

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# total schools
total_schools = school_data_complete_df["school_name"].nunique()

# total num students
total_students = len(school_data_complete_df)

# total budget
total_budget = school_data_df["budget"].sum()

# average math score
ave_math = school_data_complete_df["math_score"].mean()

# average reading score
ave_reading = school_data_complete_df["reading_score"].mean()

# percent students with passing math score (70 or greater)
total_passing_math = len(school_data_complete_df.loc[school_data_complete_df["math_score"] >= 70, :])
percent_passing_math = total_passing_math / total_students * 100

# percent students with passing reading score (70 or greater)
total_passing_reading = len(school_data_complete_df.loc[school_data_complete_df["reading_score"] >= 70, :])
percent_passing_reading = total_passing_reading / total_students * 100
percent_passing_reading

# percent students with passing math AND reading score (70 or greater) - Calculate percent overall passing
total_passing_math_and_reading = len(school_data_complete_df.loc[ (school_data_complete_df["reading_score"] >= 70) &
                                                              (school_data_complete_df["math_score"] >= 70) ])
percent_passing_math_and_reading = total_passing_math_and_reading / total_students * 100

# create a datagframe to hold all of the above data
summary_df = pd.DataFrame({"Total Schools" : [total_schools],
                          "Total Students" : f'{total_students:,}',
                          "Total Budget" : f'${total_budget:,}',
                          "Average Math Score" : f'%{ave_math:.2f}',
                          "Average Reading Score" : f'%{ave_reading:.2f}',
                          "% Passing Math" : f'%{percent_passing_math:.2f}',
                          "% Passing Reading" : f'%{percent_passing_reading:.2f}',
                          "% Overall Passing" : f'%{percent_passing_math_and_reading:.2f}'})
summary_df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",%78.99,%81.88,%74.98,%85.81,%65.17


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
# First grab the columns we need directly from the school dataframe
school_summary_df = school_data_df[["school_name", "type", "size", "budget"]]
school_summary_df = school_summary_df.rename(columns={"school_name": "School Name", "type": "School Type", "size": "Total Students", "budget": "Total School Budget"})
school_summary_df = school_summary_df.set_index("School Name")

# Need to divide the Budget column by the Students column for the Per Student Budget
school_summary_df['Per Student Budget'] = school_summary_df["Total School Budget"] / school_summary_df["Total Students"]

# Calculate average math and reading scores
# Students df --> Group by school --> average of math / reading scores
students_grouped_school = student_data_df.groupby('school_name')
means_df = students_grouped_school.mean()
# put the columns into the school summary df
school_summary_df['Average Math Score'] = means_df['math_score']
school_summary_df['Average Reading Score'] = means_df['reading_score']

school_summary_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score
School Name,,,,,,
Huang High School,District,2917,1910635,655.0,76.629414,81.182722
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020
Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757


In [4]:
# Percent passing reading column

# filter out passers from the student_data_df
passing_reading_students_df = student_data_df.loc[student_data_df['reading_score'] >= 70, :]
passing_reading_students_df = passing_reading_students_df.set_index("school_name")

# group by school
passing_reading_group = passing_reading_students_df.groupby('school_name')

# create a column for the % passing reading
school_summary_df['% Passing Reading'] = ""

# iterate over groups
for school, group in passing_reading_group:
    percent_pass = passing_reading_students_df.loc[school, "student_name"].count() / school_summary_df.loc[school, "Total Students"] * 100
    school_summary_df.at[school, '% Passing Reading'] = percent_pass 


In [5]:
# Percent passing MATH column
# filter out passers from the student_data_df
passing_math_students_df = student_data_df.loc[student_data_df['math_score'] >= 70, :]
passing_math_students_df = passing_math_students_df.set_index("school_name")

# group by school
passing_math_group = passing_math_students_df.groupby('school_name')

# create a column for the % passing reading
school_summary_df['% Passing Math'] = ""

# iterate over groups
for school, group in passing_math_group:
    percent_pass = passing_math_students_df.loc[school, "student_name"].count() / school_summary_df.loc[school, "Total Students"] * 100
    school_summary_df.at[school, '% Passing Math'] = percent_pass 
    

In [6]:
# ADD COLUMN TO SCHOOL SUMMARY DF FOR % OVERALL PASSING STUDENTS
school_summary_df['% Overall Passing'] = ""

# Filter out passers of both math and reading
passing_math_reading_students_df = student_data_df.loc[ (student_data_df['math_score'] >= 70) & 
                                                       (student_data_df["reading_score"] >= 70), :]

# Set index to the school
passing_math_reading_students_df = passing_math_reading_students_df.set_index("school_name")

# group by school
passing_math_reading_group = passing_math_reading_students_df.groupby('school_name')

# iterate over groups
for school, group in passing_math_reading_group:
    percent_pass = passing_math_reading_students_df.loc[school, "student_name"].count() / school_summary_df.loc[school, "Total Students"] * 100
    school_summary_df.at[school, '% Overall Passing'] = percent_pass 



In [7]:
# TODO Format the dataframe
school_summary_df = school_summary_df.sort_values("School Name")

school_summary_df['Total School Budget'] = school_summary_df['Total School Budget'].map('${:,.2f}'.format)
school_summary_df['Per Student Budget'] = school_summary_df['Per Student Budget'].map('${:,.2f}'.format)

school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Reading,% Passing Math,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,81.9333,66.6801,54.6423
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,97.0398,94.1335,91.3348
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,80.7392,65.9885,53.2045
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,79.299,68.3096,54.2899
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,97.139,93.3924,90.5995
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,80.863,66.753,53.5275
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,96.2529,92.5059,89.2272
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,81.3164,65.6839,53.5139
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,81.2224,66.0576,53.5392


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [11]:
# Sort by % overall passing
school_summary_sorted_df = school_summary_df.sort_values("% Overall Passing", ascending=False)
top_passing_df = school_summary_sorted_df.head()
top_passing_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Reading,% Passing Math,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,97.0398,94.1335,91.3348
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,97.3089,93.2722,90.948
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,97.139,93.3924,90.5995
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,96.5396,93.8677,90.5826
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,95.9459,94.5946,90.5405


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [12]:
# Sort by % overall passing
school_summary_sorted_df = school_summary_df.sort_values("% Overall Passing", ascending=True)
bottom_passing_df = school_summary_sorted_df.head()
bottom_passing_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Reading,% Passing Math,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,80.2201,66.3666,52.9882
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,80.7392,65.9885,53.2045
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,81.3164,65.6839,53.5139
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,80.863,66.753,53.5275
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,81.2224,66.0576,53.5392


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type